In [1]:
my_input = None
with open("input.txt") as file:
    my_input = file.read()

In [2]:
import math

def parse_tile(tile_str):
    tile = tile_str.strip().split("\n")
    tile_id = int(tile[0][5:-1])
    tile_data = tuple(tile[1:])
    return tile_id, tile_data

def parse_input(input_str):
    tiles_str = input_str.strip().split("\n\n")
    tiles = {tile_id: tile_data for tile_id, tile_data in (parse_tile(tile_str) for tile_str in tiles_str)}
    return tiles

def get_tile_edges(tile_data):
    edges = tuple([tile_data[0], "".join(row[-1] for row in tile_data), tile_data[-1], "".join(row[0] for row in tile_data)])
    return edges + tuple(edge[::-1] for edge in edges)

def map_tile_edges(tiles):
    edges_map = dict()
    for tile_id, tile_data in tiles.items():
        for edge in get_tile_edges(tile_data):
            if edge in edges_map:
                edges_map[edge] += [tile_id]
            else:
                edges_map[edge] = [tile_id]
    return edges_map


In [3]:
def part1(input_str):
    tiles = parse_input(input_str)  
    edges_map = map_tile_edges(tiles)
    edges_tile_id = [tiles[0] for tiles in edges_map.values() if len(tiles) == 1]
    corners_tile_id = {tile_id for tile_id in edges_tile_id if edges_tile_id.count(tile_id) == 4}
    return math.prod(corners_tile_id)

%time part1(my_input)

Wall time: 3 ms


51214443014783

In [28]:
tiles = parse_input(my_input)
edges_map = map_tile_edges(tiles)
edges_tile_id = tuple(tiles[0] for tiles in edges_map.values() if len(tiles) == 1)
corners_tile_id = {tile_id for tile_id in edges_tile_id if edges_tile_id.count(tile_id) == 4}

# Constant variable untuk arah penyusunan (direction) potongan gambar.

up = 0
right = 1
down = 2
left = 3

# Map dictionary untuk transpose potongan gambar.

transpose = {
        "e": lambda x: x,
        "r1": lambda x: tuple("".join(row) for row in zip(*x[::-1])),
        "r2": lambda x: tuple("".join(row[::-1]) for row in x[::-1]),
        "r3": lambda x: tuple("".join(row) for row in zip(*(row[::-1] for row in x))),
        "h": lambda x: tuple(x[::-1]),
        "v": lambda x: tuple("".join(row[::-1]) for row in x),
        "d1": lambda x: tuple("".join(row) for row in zip(*x)),
        "d2": lambda x: tuple("".join(row) for row in zip(*(row[::-1] for row in x[::-1])))
    }

# Fungsi untuk membantu mengubah posisi selanjutnya.

def move_position(position, direction):
    row, col = position
    if direction == up:
        return (row - 1, col)
    if direction == right:
        return (row , col + 1)
    if direction == down:
        return (row + 1, col)
    if direction == left:
        return (row , col - 1)

# Fungsi untuk melakukan transpose potongan gambar hingga sesuai dengan potongan gambar sebelumnya.

def transpose_tile_fit(tile_data, edge, direction):
    opp_direction = (direction + 2) % 4
    for t in transpose:
        t_tile_data = transpose[t](tile_data)
        if get_tile_edges(t_tile_data)[opp_direction] == edge:
            return t_tile_data

# Fungsi untuk menghilangkan pinggiran dari potongan gambar.

def strip_tile_edge(tile_data):
    return tuple(row[1:-1] for row in tile_data[1:-1])

# Memetakan posisi potongan gambar dimulai dengan salah satu potongan gambar yang ada di sudut.

position = (0, 0)
direction = up
current_tile_id = list(corners_tile_id)[0]
tile_data = tiles[current_tile_id]
positioned_tile = {position: strip_tile_edge(tile_data)}
used_tile_id = [current_tile_id]
retry_count = 0
while True:
    edge = get_tile_edges(tile_data)[direction]
    next_tile_id = [x for x in edges_map[edge] if not x in used_tile_id]
    if not next_tile_id:
        direction = (direction + 1) % 4
        retry_count += 1
        if retry_count >= 4:
            break
        continue
    retry_count = 0
    current_tile_id = next_tile_id[0]
    tile_data = transpose_tile_fit(tiles[current_tile_id], edge, direction)
    position = move_position(position, direction)
    positioned_tile[position] = strip_tile_edge(tile_data)
    used_tile_id.append(current_tile_id)

# Menggabungan semua potongan gambar menjadi satu

min_row = min(row for row, col in positioned_tile)
max_row = max(row for row, col in positioned_tile)
min_col = min(col for row, col in positioned_tile)
max_col = max(col for row, col in positioned_tile)
combined_tile_data = ()
for row in range(min_row, max_row + 1):
    arr = None
    for col in range(min_col, max_col + 1):
        tile_data = positioned_tile[(row, col)]
        if arr == None :
            arr = tile_data
        else:
            arr = tuple( arr[i] + tile_data[i] for i in range(len(arr)))
    combined_tile_data += arr

combined_tile_data
combined_rows_count = len(combined_tile_data)
combined_cols_count = len(combined_tile_data[0])
combined_sharps_count = "\n".join(combined_tile_data).count("#")

pattern = [
"                  # ",
"#    ##    ##    ###",
" #  #  #  #  #  #   ",
]
pattern_rows_count = len(pattern)
pattern_cols_count = len(pattern[0])
pattern_sharps_count = "\n".join(pattern).count("#")
pattern_pos = [(y, x) for y in range(pattern_rows_count) for x in range(pattern_cols_count) if pattern[y][x] == "#"]

found_monster_count = 0
for t in transpose:
    t_combined_tile_data = transpose[t](combined_tile_data)
    for i in range(combined_rows_count - pattern_rows_count):
        for j in range(combined_cols_count - pattern_cols_count):
            if all(t_combined_tile_data[i+y][j+x] == "#" for y, x in pattern_pos):
                found_monster_count += 1
    if found_monster_count:
        break

combined_sharps_count - (found_monster_count * pattern_sharps_count)


2065